In [1]:
%pylab inline
from papers import *

Populating the interactive namespace from numpy and matplotlib


In [2]:
pl, ts = arxiv_reading()
with open("./list_pkls/"+ts.split(",")[-1].strip().replace(" ", "_")+"_paper_list.pkl", "wb") as f:
    pck.dump(pl, f)
    
lp_c = ListPapers(pl)

New submissions for Mon,  7 Oct 19
Sleep...
Awake...


In [3]:
lp_c.summary()

Number of papers: 36
Subjects: 
['Solar and Stellar Astrophysics (astro-ph.SR)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Fluid Dynamics (physics.flu-dyn)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'General Relativity and Quantum Cosmology (gr-qc)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'Quantum Physics (quant-ph)', 'High Energy Physics - Theory (hep-th)', 'High Energy Physics - Experiment (hep-ex)', 'Instrumentation and Detectors (physics.ins-det)']
Key words: 

Exclude key words: 
Total scores:



In [4]:
lp_c.add_key_words(["Chemistry", "Galactic center", "SNR", "Supernova Remnant"])
lp_c.add_key_words("Cosmic Ray", boost=0.5)
lp_c.summary()


Number of papers: 36
Subjects: 
['Solar and Stellar Astrophysics (astro-ph.SR)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Fluid Dynamics (physics.flu-dyn)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'Earth and Planetary Astrophysics (astro-ph.EP)', 'General Relativity and Quantum Cosmology (gr-qc)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'Quantum Physics (quant-ph)', 'High Energy Physics - Theory (hep-th)', 'High Energy Physics - Experiment (hep-ex)', 'Instrumentation and Detectors (physics.ins-det)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'Cosmic Ray']
Exclude key words: 
Total scores:
[0.0, 0.0, 2.8727636354391248, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.337844162970578, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0729870113865014, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [5]:
lp_c.head()

ID: arXiv:1910.01668
Title: Evolving Supernova Remnants in Multiphase Interstellar Media
Authors: Marco A. Villagran; P.F. Velázquez; D.O. Gómez; E.B. Giacani
Date: New submissions for Mon,  7 Oct 19
Abstract: We performed three-dimensional magnetohydrodynamic simulations to study the
evolution of a supernova remnant (SNR) in a turbulent neutral atomic
interstellar medium. The media used as background shares characteristics with
the Solar neighbourhood and the SNR has mass and energy similar to those of a
Type Ia object. Our initial conditions consist of dense clouds in a diluted
medium, with the main difference between simulations being the average
magnitude of the magnetic field. We measured amplifications of the magnetic
energy of up to 34$\%$ and we generated synthetic maps that illustrate how the
same object can show different apparent geometries and physical properties when
observed through different lines of sight.
Comments: 
Link: 
Subjects: Astrophysics of Galaxies (astro-ph.G

In [6]:
lp_d = lp_c.filter_subjects('Earth and Planetary Astrophysics (astro-ph.EP)', exclude=True)
lp_d.summary()

Number of papers: 32
Subjects: 
['Solar and Stellar Astrophysics (astro-ph.SR)', 'Instrumentation and Methods for Astrophysics (astro-ph.IM)', 'Fluid Dynamics (physics.flu-dyn)', 'Astrophysics of Galaxies (astro-ph.GA)', 'Cosmology and Nongalactic Astrophysics (astro-ph.CO)', 'General Relativity and Quantum Cosmology (gr-qc)', 'High Energy Astrophysical Phenomena (astro-ph.HE)', 'Quantum Physics (quant-ph)', 'High Energy Physics - Theory (hep-th)', 'High Energy Physics - Experiment (hep-ex)', 'Instrumentation and Detectors (physics.ins-det)']
Key words: 
['Chemistry', 'Galactic center', 'SNR', 'Supernova Remnant', 'Cosmic Ray']
Exclude key words: 
Total scores:
[0.0, 0.0, 2.7658708857048588, 0.0, 0.0, 0.0, 0.0, 0.0, 8.9705103212306021, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.9393021906123384, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [7]:
lp_d.head(sync=True)

ID: arXiv:1910.01668
Title: Evolving Supernova Remnants in Multiphase Interstellar Media
Authors: Villagran, Marco A.; Velázquez, P. F.; Gómez, D. O.; Giacani, E. B.
Date: 2019/10/03
Abstract: We performed three-dimensional magnetohydrodynamic simulations to study the
evolution of a supernova remnant (SNR) in a turbulent neutral atomic
interstellar medium. The media used as background shares characteristics with
the Solar neighbourhood and the SNR has mass and energy similar to those of a
Type Ia object. Our initial conditions consist of dense clouds in a diluted
medium, with the main difference between simulations being the average
magnitude of the magnetic field. We measured amplifications of the magnetic
energy of up to 34$\%$ and we generated synthetic maps that illustrate how the
same object can show different apparent geometries and physical properties when
observed through different lines of sight.
Comments: 
Link: https://arxiv.org/abs/arXiv:1910.01668
Subjects: Astrophysics of

In [8]:
print(lp_c._boost)

{'chemistry': 1.0, 'galactic center': 1.0, 'snr': 1.0, 'supernova remnant': 1.0, 'cosmic ray': 0.5}


In [12]:
lp_c[0].arxiv_id

'arXiv:1910.01643'

In [13]:
dir(lp_c)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_boost',
 '_idf',
 '_kw_match',
 '_kw_n',
 '_subj_contain',
 '_tfnorm',
 '_update_tot_scores',
 'add_key_words',
 'all_subjects',
 'aver_length',
 'cal_key_word_scores',
 'del_key_word',
 'exclude_key_words',
 'filter_subjects',
 'head',
 'key_words',
 'list_paper',
 'reset_exclude',
 'scores',
 'search_keyword',
 'subjects_list',
 'summary',
 'tot_num',
 'tot_scores']